### Collecting Fashion House Tweets

In [21]:
import time, pandas as pd, numpy as np

In [1]:
#Reading in application keys 

with open('../data/twitter_keys.txt') as api:
    keys = api.read().split('\n') 

#Aplication keys

import twitter, re, datetime, pandas as pd, numpy as np

twitter_keys = {
    'consumer_key':        keys[0],
    'consumer_secret':     keys[1],
    'access_token_key':    keys[2],
    'access_token_secret': keys[3]
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret'],
    tweet_mode           =   'extended'
)


In [2]:
#Twitter API Class

class TweetMiner():
    
    result_limit = 20
    api          = None
    tweets       = []
    
    def __init__(self, keys_dict, api, result_limit = 20):
        
        self.keys_dict     = keys_dict
        self.api           = api
        self.results_limit = result_limit
        
    def mine_user_tweets(self, user='gucci', mine_retweets = False, max_pages = None, last_tweet_id = False):
        
        #time out feedback
        print("Mining tweets for:", user)
        
        tweets        = []
        
        page           = 1
        
        while page <= max_pages:
            
            #time out feedback
            print("Mining page:", page)
            
            if last_tweet_id:
                statuses = self.api.GetUserTimeline(
                    screen_name = user,
                    count       = self.result_limit,
                    max_id      = last_tweet_id - 1
                )
            else:
                statuses = self.api.GetUserTimeline(
                    screen_name = user,
                    count       = self.result_limit
                )

            for item in statuses:
                
                last_tweet_id = item.id
                
                tweets.append({
                    'tweet_id': item.id,
                    'handle':   item.user.name,
                    'text':     item.full_text,
                    'created':  item.created_at
                })
                
            page += 1
        
        return tweets
            
        
miner = TweetMiner(twitter_keys, api, result_limit=50)

### Fashion Labels

In [13]:
wikipedia_labels = ['akris_official', 'McQueen', 'alfreddunhill', 'AlphaIndustries', 'andres_sarda', 'Anne_Fontaine',\
                   'aquascutum', 'giorgioarmani', 'emporioarmani', 'ArmaniExchange', 'AspinalofLondon', 'AspreyLondon',\
                   'AleDellAcqua', 'Dior', 'dkny', 'dolcegabbana', '_BadgleyMischka', 'BadgleyMischka', 'BALENCIAGA',\
                   'Bally', 'Balmain', 'BensonAndClegg', 'berluti', 'Beulahlondon', 'ManoloBlahnik', 'HUGOBOSS',\
                    'BottegaVeneta', 'ThomBrowneNY', 'Bulgariofficial', 'Burberry', 'NigelCabourn', 'CalvinKlein',\
                   'camillaandmarc', 'Cartier', 'Dsquared2', 'CERRUTI1881', 'CHANEL', 'Childrensalon', 'chloefashion',\
                   'matthewbridal', 'chromeheartsusa', 'ClementsRibeiro', 'Coach', 'COMMEGARCONS', 'Common Projects',\
                   'corneliajames', 'Corneliani_com', 'CuteCircuit', 'cutlerandgross', 'DegeandSkinner', 'NUMEROVENTUNO',\
                   'dkny', 'dolcegabbana', 'ermannoscervino', 'ESCADA', 'EtroOfficial', 'Faconnable', 'Fendi',\
                    'TOMFORD', 'franksorbier', 'freywilleglobal', 'STUDIO_FULTON', 'DVF', 'JPGaultier', 'GievesLondon',\
                   'givenchy', 'Goyard', 'gucci', 'HackettLondon', 'KevanHallDesign', 'halston', 'Hardy Amies ', 'HELMUTLANG',\
                   'Hermes_Paris', 'HouseofHerrera', 'TommyHilfiger', 'SherriHill', 'GeorgesHobeika', 'marcjacobs', 'jimmychoo',\
                   'johnvarvatos', 'ChristopherKane', 'Karen_Kane', 'katespadeny', 'MaryKatrantzou', ' Khaadi', 'LACOSTE',\
                   'BNYhandbags', 'KarlLagerfeld', 'DerekLamNYC', 'LANVINofficial', 'LardiniOfficial', 'guylaroche', 'Larusmiani',\
                   'JudithLeiber', 'M_Lhuillier', '31philliplim', 'DanLiuTatsuaki', 'Longchamp', 'LouboutinWorld',\
                   'Luxottica', 'BrunoMagli', 'Margiela', 'MarchesaFashion', 'Marimekkoglobal', 'marniofficial',\
                   'maxmara', 'StellaMcCartney', 'MCMtweets', 'MichaelKors', 'Missoni', 'MIUMIUofficial', 'Moschino',\
                   'mouawadjewelry', 'MoynatParis', 'Mugler', 'MulberryEngland', 'charlottes_web', 'RICKOWENSONLINE',\
                   'PauleKa', 'philipp_plein', 'Prada', 'PringleScotland', 'EmilioPucci', 'PacoRabanne', 'Raja Fashions',\
                   'ralphandrusso', 'RalphLauren', 'OscardelaRenta', 'NinaRicci', 'Roberto_Cavalli', 'OfficialRodarte',\
                   'narcisostudio', 'RolandMouret', 'sergiorossi', 'MrRalphRucci', 'soniarykiel', '']


fashion_labels = ['gucci', 'McQueen', 'StellaMcCartney', 'Fendi', 'MaisonValentino', 'Roberto_Cavalli',\
                  'LouboutinWorld', 'EtroOfficial', 'Prada', 'dolcegabbana', 'Dior', 'LouisVuitton', 'CHANEL'\
                 'Missoni', 'OscardelaRenta', 'VeraWangGang', 'HerveLeger', 'MaisonValentino', 'Balmain',\
                 'Versace', 'giorgioarmani', 'DerekLamNYC', 'AlbertaFerretti', 'YSL', 'COMMEGARCONS', 'BALENCIAGA',\
                 'Burberry', 'TOMFORD', 'ALEXVAUTHIER', 'MIUMIUofficial', 'CalvinKlein', 'givenchy', 'TommyHilfiger',\
                 'jimmychoo', 'OffWht', 'chloefashion','dkny', 'marcjacobs', 'MichaelKors', 'Goyard', 'HUGOBOSS',\
                 'kenzo', 'HELMUTLANG', 'acnestudios', 'Bulgariofficial', 'akris_official']

prime_labels = ['gucci', 'McQueen', 'StellaMcCartney', 'Roberto_Cavalli', 'acnestudios', 'jimmychoo', 'Roberto_Cavalli',\
               'TOMFORD', 'Prada', 'akris_official']

print('wikipedia_labels', len(wikipedia_labels))
print('fashion_labels:', len(fashion_labels))
print('prime_labels:', len(prime_labels))

wikipedia_labels 95
fashion_labels: 45
prime_labels: 10


### Collecting and storing data

In [ ]:
# scraper is not working yet


def scrape_label(label):
    try:
        tweet_data = miner.mine_user_tweets(user=label, max_pages=200, last_tweet_id=True)
        df = pd.DataFrame(tweet_data)
        df.to_csv('../Tweets/%s.csv' % label)
    except:
        print('Timed out on:', label)
        with open('../Tweets/logfile.txt', 'a+') as f:
            f.write(f'Timed out on {label} at {time.ctime()}\n')
        time.sleep(945)
        scrape_label(label)
        
for label in prime_labels:
    scrape_label(label)         

In [ ]:
reversed()

In [3]:
#test

tweets = miner.mine_user_tweets(user='gucci', max_pages=500)

Mining tweets for: gucci
Mining page: 1
Mining page: 2
Mining page: 3
Mining page: 4
Mining page: 5
Mining page: 6
Mining page: 7
Mining page: 8
Mining page: 9
Mining page: 10
Mining page: 11
Mining page: 12
Mining page: 13
Mining page: 14
Mining page: 15
Mining page: 16
Mining page: 17
Mining page: 18
Mining page: 19
Mining page: 20
Mining page: 21
Mining page: 22
Mining page: 23
Mining page: 24
Mining page: 25
Mining page: 26
Mining page: 27
Mining page: 28
Mining page: 29
Mining page: 30
Mining page: 31
Mining page: 32
Mining page: 33
Mining page: 34
Mining page: 35
Mining page: 36
Mining page: 37
Mining page: 38
Mining page: 39
Mining page: 40
Mining page: 41
Mining page: 42
Mining page: 43
Mining page: 44
Mining page: 45
Mining page: 46
Mining page: 47
Mining page: 48
Mining page: 49
Mining page: 50
Mining page: 51
Mining page: 52
Mining page: 53
Mining page: 54
Mining page: 55
Mining page: 56
Mining page: 57
Mining page: 58
Mining page: 59
Mining page: 60
Mining page: 61
Mining p

Mining page: 489
Mining page: 490
Mining page: 491
Mining page: 492
Mining page: 493
Mining page: 494
Mining page: 495
Mining page: 496
Mining page: 497
Mining page: 498
Mining page: 499
Mining page: 500


In [4]:
tweets

[{'created': 'Wed Mar 28 17:45:08 +0000 2018',
  'handle': 'gucci',
  'text': 'Attending a private dinner at #BiboHK to celebrate @parrstudio’s first project with the House: Nydia Zhang, Feiping Chang and @bryanboy. #TimeToParr #GucciTimepieces #GucciPlaces\nTM &amp; © Warner Bros. Entertainment Inc. (s18) https://t.co/JeygdwdWQc',
  'tweet_id': 979051778678448128},
 {'created': 'Wed Mar 28 16:41:35 +0000 2018',
  'handle': 'gucci',
  'text': "Celebrating @parrstudio’s first project with the House with a private dinner at #BiboHK. Attending the event: #MartinParr, Diana d'Arenberg, @tinaleung and @susiebubble. #TimeToParr #GucciTimepieces #GucciPlaces https://t.co/h8Il6yGRTa",
  'tweet_id': 979035785633718272},
 {'created': 'Wed Mar 28 12:42:52 +0000 2018',
  'handle': 'gucci',
  'text': 'People looking at a map of @ArtBasel Hong Kong captured by #MartinParr @parrstudio. Soon to be revealed, the photographer’s first project with the House-an uncommon photographic journey involving #Guc

In [ ]:
#Instantiating class and passing list of twitter handles

for label in fashion_labels:
    try:
        tweet_data = miner.mine_user_tweets(user=label, max_pages=250, last_tweet_id=True)
        df = pd.DataFrame(tweet_data)
        df.to_csv('../data/%s.csv' % label)
    except:
        print('could not mine label:', label)


In [22]:
#reading in fashion house csv and creating master datafram

acne_df      = pd.read_csv('../data/acnestudios.csv',index_col=[0])
akris_df     = pd.read_csv('../data/akris_official.csv',index_col=[0])
alberta_df   = pd.read_csv('../data/AlbertaFerretti.csv',index_col=[0])
alexvauth_df = pd.read_csv('../data/ALEXVAUTHIER.csv',index_col=[0])
balenci_df   = pd.read_csv('../data/BALENCIAGA.csv',index_col=[0])

balmain_df   = pd.read_csv('../data/Balmain.csv',index_col=[0])
bulgari_df   = pd.read_csv('../data/Bulgariofficial.csv',index_col=[0])
burbery_df   = pd.read_csv('../data/Burberry.csv',index_col=[0])
calvin_df    = pd.read_csv('../data/CalvinKlein.csv',index_col=[0])
chanel_df    = pd.read_csv('../data/CHANEL.csv',index_col=[0])

chloe_df     = pd.read_csv('../data/chloefashion.csv',index_col=[0])
commes_df    = pd.read_csv('../data/COMMEGARCONS.csv',index_col=[0])
dereklam_df  = pd.read_csv('../data/DerekLamNYC.csv',index_col=[0])
dior_df      = pd.read_csv('../data/Dior.csv',index_col=[0])
dkny_df      = pd.read_csv('../data/dkny.csv',index_col=[0])

dolce_df     = pd.read_csv('../data/dolcegabbana.csv',index_col=[0])
etro_df      = pd.read_csv('../data/EtroOfficial.csv',index_col=[0])
fendi_df     = pd.read_csv('../data/Fendi.csv',index_col=[0])
armani_df    = pd.read_csv('../data/giorgioarmani.csv',index_col=[0])
givenchy_df  = pd.read_csv('../data/givenchy.csv',index_col=[0])

goyard_df    = pd.read_csv('../data/Goyard.csv',index_col=[0])
gucci_df     = pd.read_csv('../data/gucci.csv',index_col=[0])
helmut_df    = pd.read_csv('../data/HELMUTLANG.csv',index_col=[0])
herve_df     = pd.read_csv('../data/HerveLeger.csv',index_col=[0])
hugoboss_df  = pd.read_csv('../data/HUGOBOSS.csv',index_col=[0])

jimmychoo_df = pd.read_csv('../data/jimmychoo.csv',index_col=[0])
lagerfeld_df = pd.read_csv('../data/KarlLagerfeld.csv',index_col=[0])
kenzo_df     = pd.read_csv('../data/kenzo.csv',index_col=[0])
lanvin_df    = pd.read_csv('../data/LANVINofficial.csv',index_col=[0])
Louboutin_df = pd.read_csv('../data/LouboutinWorld.csv',index_col=[0])

louisv_df   = pd.read_csv('../data/LouisVuitton.csv',index_col=[0])
valentino_df = pd.read_csv('../data/MaisonValentino.csv',index_col=[0])
marcjacob_df = pd.read_csv('../data/marcjacobs.csv',index_col=[0])
mcqueen_df   = pd.read_csv('../data/McQueen.csv',index_col=[0])
kors_df      = pd.read_csv('../data/MichaelKors.csv',index_col=[0])

missoni_df   = pd.read_csv('../data/Missoni.csv',index_col=[0])
miumiu_df    = pd.read_csv('../data/MIUMIUofficial.csv',index_col=[0])
offwht_df    = pd.read_csv('../data/OffWht.csv',index_col=[0])
oscar_df     = pd.read_csv('../data/OscardelaRenta.csv',index_col=[0])
prada_df     = pd.read_csv('../data/Prada.csv',index_col=[0])

cavalli_df   = pd.read_csv('../data/Roberto_Cavalli.csv',index_col=[0])
stella_df    = pd.read_csv('../data/StellaMcCartney.csv',index_col=[0])
tomford_df   = pd.read_csv('../data/TOMFORD.csv',index_col=[0])
hilfiger_df  = pd.read_csv('../data/TommyHilfiger.csv',index_col=[0])
verawang_df  = pd.read_csv('../data/VeraWangGang.csv',index_col=[0])

versache_df  = pd.read_csv('../data/Versace.csv',index_col=[0])
ysl_df       = pd.read_csv('../data/YSL.csv',index_col=[0])


df_list = [acne_df, akris_df, alberta_df, alexvauth_df, balenci_df, balmain_df, bulgari_df, burbery_df, calvin_df,\
           chanel_df, chloe_df, commes_df, dereklam_df, dior_df, dkny_df, dolce_df, etro_df, fendi_df,\
           armani_df, givenchy_df, goyard_df, gucci_df, helmut_df, herve_df, hugoboss_df, jimmychoo_df,\
           lagerfeld_df, kenzo_df, lanvin_df, Louboutin_df, louisv_df, valentino_df, marcjacob_df, mcqueen_df,\
           kors_df, missoni_df, miumiu_df, offwht_df, oscar_df, prada_df, cavalli_df, stella_df, tomford_df,\
           hilfiger_df, verawang_df, versache_df, ysl_df]

fashion_df = pd.concat(df_list, ignore_index=True)
fashion_df.shape

(113881, 4)

In [23]:
fashion_df.groupby(fashion_df.handle).size().sort_values(ascending=False)

handle
Roberto Cavalli        3247
Dolce & Gabbana        3245
Vera Wang              3244
Valentino              3240
Stella McCartney       3238
Missoni                3238
Oscar de la Renta      3234
Michael Kors           3234
VERSACE                3232
HUGO BOSS              3231
gucci                  3231
Marc Jacobs            3226
CALVIN KLEIN           3225
Fendi                  3224
AlbertaFerretti        3221
COMME des GARÇONS      3219
ETRO                   3217
Burberry               3216
Giorgio Armani         3212
Alexander McQueen      3209
Louis Vuitton          3209
Tommy Hilfiger         3205
Christian Louboutin    3204
Balmain                3204
Off-White™             3175
Derek Lam              3014
Bulgari                2853
Chloé                  2813
LANVIN                 2704
Dior                   2567
PRADA                  2005
KARL LAGERFELD         1978
CHANEL                 1819
Herve Leger            1409
DKNY                   1278
MIU MIU      

In [24]:
fashion_df.to_pickle('../data/fashion_df.pkl')